In [1]:
%display latex

%runfile diagonal_polynomial_ring.py
%runfile subspace.py
%runfile young_idempotent.py
%runfile add_degree.py
%runfile diagram.py

SymmetricFunctions(QQ).inject_shorthands(verbose=False)

#%runfile compute_character.py

# Code

Calcul du Vandermonde associé au diagramme $\gamma$, où $\gamma$ est un partage (Partition) ou un diagramme (Diagram).

In [2]:
def vandermonde(gamma):
    n = gamma.size()
    P = DiagonalPolynomialRing(QQ, n, 1, inert=1)
    X = P.variables()
    Theta = P.inert_variables()
    return matrix([[x**i[1]*theta**i[0] for i in gamma.cells()] 
                   for x,theta in zip(X[0],Theta[0])]).determinant()

def degree_vandermonde(gamma):
    return sum([i[1] for i in gamma.cells()])

Fonctions qui nous renvoient des listes d'opérateurs par degrés pour toutes les variables (non inertes) de l'anneau de polynômes concerné.

In [3]:
def deriv(x, k=1):
    def f(p):
        return derivative(p, x, k)
    return f

def partial_derivatives(P):
    n = P._n
    r = P._r
    D = P._grading_set
    X = P.variables()
    op = {}
    for i in range(r):
        op[D((-1 if j==i else 0 for j in range(r)))] = [deriv(X[i,k]) for k in range(n)]
    return op

def steenrod_operators(P, degree=1, row_symmetry=None):
    r = P._r
    D = P._grading_set
    op = {}
    for i in range(r):
        op[D((-degree if j==i else 0 for j in range(r)))] = [
            functools.partial(P.steenrod_op, i=i, k=degree+1, row_symmetry=row_symmetry)]
    return op

def diagonal_steenrod_operators(P, list_deg):
    r = P._r
    D = P._grading_set
    op = {}
    for dx, dy in list_deg:
        op[D((-dx if j==0 else -dy if j==1 else 0 for j in range(r)))] = [
            functools.partial(P.diagonal_steenrod, i1=0, i2=1, d1=dx, d2=dy)]
    return op

def polarization_operators(P, side=None, row_symmetry=None, max_deg=0):
    n = P._n
    r = P._r
    D = P._grading_set
    if max_deg==0:
        max_deg=n
    return {D([-d if i==i1 else 1 if i==i2 else 0 for i in range(r)]):
            [functools.partial(P.polarization, i1=i1, i2=i2, d=d, row_symmetry=row_symmetry)]
            for d in range(1, max_deg+1)
            for i1 in range(0, r)
            for i2 in range(0, r)
            if (i1<i2 if side == 'down' else i1!=i2)
           }

def higher_polarization_operators(P, side=None, row_symmetry=None, max_deg=0):
    n = P._n
    r = P._r
    D = P._grading_set
    if max_deg==0:
        max_deg=n
    return {D([-d1 if i==i1 else d2 if i==i2 else 0 for i in range(r)]):
            [functools.partial(P.higher_polarization, i1=i1, i2=i2, d1=d1, d2=d2, row_symmetry=row_symmetry)]
            for d1 in range(1, max_deg+1)
            for d2 in range(1, 3)
            for i1 in range(0, r)
            for i2 in range(0, r)
            if (i1<i2 if side == 'down' else i1!=i2)
           }

def multipolarization(P, list_deg, i2): 
    D = P._grading_set
    return {D(-d[i]+1 if i==i2 else -d[i] for i in range(len(d))) : 
            [functools.partial(P.multi_polarization, D=d, i2=i2)] for d in list_deg}

def symmetric_derivatives(P, list_deg, row_symmetry=None):
    D = P._grading_set
    return {D(-i for i in d) : [functools.partial(P.symmetric_derivative, d=d, row_symmetry=row_symmetry)] 
            for d in list_deg}

In [4]:
def merge(dict1, dict2):
    result = dict1
    for key, value in dict2.iteritems():
        if key in result:
            result[key] += value
        else:
            result[key] = value
    return result

def antisymmetries(mu):
    mu = Partition(mu)
    return antisymmetries_of_tableau(mu.initial_tableau())

Projection sur les composantes isotypiques; prend en paramètre un sous-espace (Subspace) et un entier ou un partage. 

In [5]:
def Isotyp(S, arg, use_antisymmetry=False):
    if isinstance(arg, Partition):
        list_partitions = [arg]
    elif isinstance(arg, Integer):
        list_partitions = Partitions(arg)
    else : 
        print("Error: arg should be a partition or an integer.")
    
    basis = S.basis()
    result = {}
    for nu in list_partitions:
        for key, value in basis.iteritems():
            gen = [apply_young_idempotent(p, nu, use_antisymmetry=use_antisymmetry) for p in value]
            basis_nu = Subspace(gen, {}).basis()
            if basis_nu != {} :
                result[(key, tuple(nu))] = basis_nu[0]
    if result:
        return Subspace(result, operators={})
    else:
        return Subspace([0],{})

def Isotyp2(S, arg):
    if isinstance(arg, Partition):
        list_partitions = [arg]
    elif isinstance(arg, Integer):
        list_partitions = Partitions(arg)
    else : 
        print("Error: arg should be a partition or an integer.")
    
    basis = S.basis()
    result = {}
    for nu in list_partitions:
        for key, value in basis.iteritems():
            gen = [apply_young_idempotent(p, nu) for p in value]
            basis_nu = Subspace(gen, {}).basis()
            if basis_nu != {} :
                result[(key[0], tuple(nu))] = basis_nu[0] #key[0] >>> key
    return Subspace(result, operators={})

In [6]:
def add_degrees_isotypic(gen_deg, op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    return D(gen_deg[0])+D(op_deg), gen_deg[1]

def add_degrees_symmetric(gen_deg,op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    d = D(gen_deg[0])+D(op_deg)
    return D(sorted(d, reverse=True)), gen_deg[1]

def add_degrees_test(gen_deg, op_deg):
    return gen_deg+op_deg

Création de l'espace polarisé par les opérateurs donnés en paramètres.

In [7]:
def PolarizedSpace(P, S, operators, add_degrees=add_degrees_isotypic):
    basis = S.basis()
    n = P._n
    r = P._r
    inert = P._inert
    D = P._grading_set
    generators = {}
    
    if isinstance(P, DiagonalAntisymmetricPolynomialRing):
        antisymmetries = P._antisymmetries
        for key, value in basis.iteritems():
            d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
            generators[d] = [antisymmetric_normal(P(b), n, r+inert, antisymmetries) for b in value]
    else :
        for key, value in basis.iteritems():
            d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
            generators[d] = [P(b) for b in value]
            
    if generators:
        return Subspace(generators, operators, add_degrees=add_degrees)
    else:
        return Subspace([0],{})

Calcul de l'image de S (Subspace) par les opérateurs donnés.

In [8]:
def Range(S, operators, add_degrees=add_degrees_isotypic):
    result = {}
    basis = S.basis()
    for key, b in basis.iteritems():
        result = merge(result, {add_degrees(key, deg): 
                                     [op(p) for p in b for op in op_list if op(p)!=0] 
                                     for deg, op_list in operators.iteritems()})    
    if result != {} :
        return Subspace(result, {}, add_degrees) #{} <-> operators
    else :
        return None

Calcul du caractère à partir d'un sous-espace (S) d'éléments projetés sur les composantes isotypiques.

In [9]:
def character(S, n, r, left_basis=s, right_basis=s, row_symmetry=None):
    basis = S.basis()
    charac = 0
    if row_symmetry != "permutation":
        q = PolynomialRing(QQ,'q',r).gens()
        
    for nu in Partitions(n):
        basis_nu = {}
        charac_nu = 0
        # Get the nu_isotypic part of the basis
        for key, value in basis.iteritems():
            if Partition(key[1]) == nu:
                basis_nu[key[0]] = value
        
        # Use monomials to compute the character
        if row_symmetry == "permutation":
            for deg, b in basis_nu.iteritems():
                charac_nu += sum(m(Partition(deg)) for p in b)
            if charac_nu != 0 :
                if left_basis == s :
                    charac_nu = s(charac_nu).restrict_partition_lengths(r,exact=False)
                elif left_basis != m :
                    charac_nu = left_basis(charac_nu)
                
        # Or use directly the degrees
        else:
            for deg, b in basis_nu.iteritems():
                charac_nu += sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
            if charac_nu != 0 :
                if left_basis == s :
                    charac_nu = s.from_polynomial(charac_nu).restrict_partition_lengths(r,exact=False)           
                else:
                    charac_nu = left_basis.from_polynomial(charac_nu)
                
        # Make the tensor product with s[nu]
        if charac_nu != 0:
            charac += tensor([charac_nu, right_basis(s(nu))])
    return charac

In [10]:
def character_quotient(M, N, n, r, left_basis=s, right_basis=s):
    b_tot = M.basis()
    b_ideal = N.basis()
    charac = 0
    q = PolynomialRing(QQ,'q',r).gens()
    
    for nu in Partitions(n):
        basis_nu_tot = {}
        basis_nu_ideal = {}
        charac_nu = 0
        # Get the nu_isotypic part of the bases
        for key, value in b_tot.iteritems():
            if Partition(key[1]) == nu:
                basis_nu_tot[key[0]] = value
        for key, value in b_ideal.iteritems():
            if Partition(key[1]) == nu:
                basis_nu_ideal[key[0]] = value
                
        # Use the degrees to compute the character
        for deg, b in basis_nu_tot.iteritems():
            charac_nu += sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
        for deg, b in basis_nu_ideal.iteritems():
            charac_nu -= sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
        if charac_nu != 0 :
            if left_basis == s :
                charac_nu = s.from_polynomial(charac_nu).restrict_partition_lengths(r,exact=False)           
            else:
                charac_nu = left_basis.from_polynomial(charac_nu)      
            # Make the tensor product with s[nu]
            charac += tensor([charac_nu, right_basis(s(nu))])
            
    return charac

Factorisation du produit tensoriel par représentation de $S_n$. Et calcul de la dimension.

In [11]:
def factorise(f, n):
    SymmetricFunctions(QQ).s()
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu])) for mu in result.keys()]
    for a, b in result2:
        if b!=0:
            print a
            show(b)
        
def dimension(f, n):
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu]).expand(1, alphabet=['q'])) for mu in result.keys()]
    q = result2[0][1].parent().gens()[0]
    return [(tuple(a), b.subs({q:1})) for a,b in result2]

def dimension2(f, n):
    q,t = QQ['q,t'].gens()
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu]).expand(2, alphabet=[q,t])) for mu in result.keys()]
    return [(tuple(a), b.subs({q:1,t:1})) for a,b in result2]

### Cas du diagramme 013

In [12]:
v = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
v.factor()

(x01 - x02) * (-x00 + x01) * (x00 - x02) * (x00 + x01 + x02)

On crée le sous-espace engendré par $\mathcal{V}_\gamma$ et ses dérivées partielles. On peut ajouter les opérateurs de Steenrod sans changement du résultat.

In [13]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
generator = {P1._grading_set((v.degree(),)) : [P1(v)]}
list_op = merge(merge(partial_derivatives(P1), steenrod_operators(P1, 1)), steenrod_operators(P1, 2))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (-x00*x01 + x00*x02, -x00*x01 + x01*x02, x00^2 - x02^2, x01^2 - x02^2),
 (3,): (1/3*x00^3 - 1/3*x01^3 - x00*x02^2 + x01*x02^2,
  -x00^2*x01 + 1/3*x01^3 + x00^2*x02 - 1/3*x02^3,
  -1/3*x00^3 + x00*x01^2 - x01^2*x02 + 1/3*x02^3),
 (4,): (x00^3*x01 - x00*x01^3 - x00^3*x02 + x01^3*x02 + x00*x02^3 - x01*x02^3,)}

In [14]:
character(Isotyp(W1, 3), 3, 1)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Appliquons $\sum_i \partial_{x_i}$ et $\sum_i x_i \partial_{x_i}^2$ à $\mathcal{V}_{\gamma}$. 

In [15]:
x00, x01, x02 = P1.variables()[0]
P1.symmetric_derivative(P1(v), (1,))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [16]:
P1.steenrod_op(P1(v), 0, 2)

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

Polarisation

In [17]:
P2 = DiagonalPolynomialRing(QQ, 3, 2)
op_pol = polarization_operators(P2)
W2 = PolarizedSpace(P2, Isotyp(W1, 3), op_pol)
character(W2, 3, 2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

In [18]:
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(P2._r)) 
                for k1 in range(1, 3+1) for i1 in range(0, P2._r)]
sym_diff = symmetric_derivatives(P2, list_degrees)
character(Range(W2, sym_diff), 3, 2)

s[] # s[3] + s[1] # s[2, 1] + s[1, 1] # s[1, 1, 1] + s[2] # s[2, 1] + s[3] # s[1, 1, 1]

In [19]:
charac = character(W2, 3, 2) - character(Range(W2, sym_diff), 3, 2)
charac

s[1] # s[3] + s[1, 1] # s[2, 1] + s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Ou dans la base des monomiales

In [20]:
character(W2, 3, 2, left_basis=m) - character(Range(W2, sym_diff), 3, 2, left_basis=m)

m[1] # s[3] + 2*m[1, 1] # s[2, 1] + m[2] # s[2, 1] + m[2, 1] # s[1, 1, 1] + m[2, 1] # s[2, 1] + m[2, 2] # s[1, 1, 1] + m[3] # s[2, 1] + m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]

On factorise le caractère par représentation de $S_3$.

In [21]:
factorise(charac, 3)

[1, 1, 1]


s[2, 1] + s[4]

[3]


s[1]

[2, 1]


s[1, 1] + s[2] + s[3]

### Cas du diagramme 024 : en cours de résolution

In [22]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
v.factor()

(x01 - x02) * (x01 + x02) * (-x00 + x01) * (x00 - x02) * (x00 + x02) * (x00 + x01)

### Tests pour 024

**Quotient puis polarisation** et test des dimensions (vérification du symétriseur de Young)

In [23]:
# Create M with V, partial derivatives and Steenrod in x
P1 = DiagonalPolynomialRing(QQ, 3, 1)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = degree_vandermonde(Diagram([(0,0),(2,0),(4,0)]))
generator = {P1._grading_set((deg,)) : [P1(v)]}
#list_op = partial_derivatives(P1)
list_op = merge(steenrod_operators(P1, 1), steenrod_operators(P1, 2))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
print "dimension V1:",W1.dimension()
W1 = Isotyp(W1, Partition([1,1,1]))
#print dimension(character(W1, 3, 1), 3)
print

# Create ideal I*M in x using symmetric derivatives
list_degrees = [tuple((i,)) for i in range(1,4)]
#sym_diff = symmetric_derivatives(P1, list_degrees)
sym_diff = merge(steenrod_operators(P1, 1), steenrod_operators(P1, 2))
W2 = Range(W1, sym_diff)

# Print caracters in one set of variables
print "V1(X)"
show(character(W1, 3, 1))
print "V2(X)"
show(character(W2, 3, 1))
print "V1(X)-V2(X)"
show(character(W1, 3, 1)- character(W2, 3, 1))
show(character_quotient(W1, W2, 3, 1))

# Polarisation of M and I*M
r=2
P2 = DiagonalPolynomialRing(QQ, 3, r)
op_pol = polarization_operators(P2, max_deg=4)
op_pol2 = polarization_operators(P2, max_deg=4, side="down")
list_degrees = [tuple((i,j)) for i in range(1,deg) for j in range(1,deg) if i+j!=0 and i+j<=deg]
list_degrees += [(0,1), (0,2), (1,0), (2,0)]
#op_pol2 = merge(op_pol2, diagonal_steenrod_operators(P2, list_degrees))
op_pol = merge(merge(op_pol, steenrod_operators(P2, 1)), steenrod_operators(P2,2))
op_pol2 = merge(merge(op_pol2, multipolarization(P2, list_degrees, 0)), multipolarization(P2, list_degrees, 1))
#op_pol = merge(op_pol, multipolarization(P2, list_degrees, 0))
#op_pol = merge(op_pol, multipolarization(P2, list_degrees, 1))
W1 = PolarizedSpace(P2, W1, op_pol)
W2 = PolarizedSpace(P2, W2, op_pol2)

# Print caracters in two sets of variables
print "V1(X,Y)"
show(character(W1, 3, r, left_basis=m))
print "V2(X,Y)"
show(character(W2, 3, r, left_basis=m))
charac = character(W1, 3, r, left_basis=m) - character(W2, 3, r, left_basis=m)
print "V1(X,Y)-V2(X,Y)"
show(charac)
show(character_quotient(W1, W2, 3, 2))

dimension V1: 4

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1]

V1(X)-V2(X)


...

V2(X,Y)


m[1, 1] # s[1, 1, 1] + 2*m[2, 1] # s[1, 1, 1] + 2*m[2, 2] # s[1, 1, 1] + m[3] # s[1, 1, 1] + 2*m[3, 1] # s[1, 1, 1] + m[3, 2] # s[1, 1, 1] + m[4] # s[1, 1, 1] + m[4, 1] # s[1, 1, 1] + m[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


m[2, 2] # s[1, 1, 1] + m[3, 1] # s[1, 1, 1] + m[3, 2] # s[1, 1, 1] + m[3, 3] # s[1, 1, 1] + m[4, 1] # s[1, 1, 1] + m[4, 2] # s[1, 1, 1] + m[5, 1] # s[1, 1, 1] + m[6] # s[1, 1, 1]

s[3, 1] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[6] # s[1, 1, 1]

**Polarisation puis quotient**

In [12]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = 6
generator = {P1._grading_set((6,)) : [P1(v)]}
list_op = partial_derivatives(P1)
list_op = merge(merge(list_op, steenrod_operators(P1, 1)), steenrod_operators(P1, 2))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1 = Isotyp(W1, Partition([1,1,1]))

print "V1(X)"
show(character(W1, 3, 1))

r=2
P2 = DiagonalPolynomialRing(QQ, 3, r)
op_pol = polarization_operators(P2, side="down")
op_pol = merge(merge(op_pol, steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
W1 = PolarizedSpace(P2, W1, op_pol)

print "V1(X,Y)"
show(character(W1, 3, r))

list_degrees = [tuple((i,j)) for i in range(0, deg) for j in range(0, deg) if i+j>0 and i+j<deg]
sym_diff = symmetric_derivatives(P2, list_degrees)
sym_diff = merge(merge(sym_diff, steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
W2 = Range(W1, sym_diff, add_degrees=add_degrees_isotypic)

print "V2(X,Y)"
show(character(W2, 3, r))

print "V1(X,Y)-V2(X,Y)"
charac = character(W1, 3, r) - character(W2, 3, r)
show(charac)

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[6] # s[1, 1, 1]

**Tests** sur le modèle polarisation puis quotient

In [40]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = 6
generator = {P1._grading_set((6,)) : [P1(v)]}
list_op2 = merge(steenrod_operators(P1, 1), steenrod_operators(P1, 2))
W1 = Subspace(generators=generator, operators=list_op2, add_degrees=add_degree)
W1 = Isotyp(W1, Partition([1,1,1]))
W1bis = Subspace(generators=generator, operators={}, add_degrees=add_degree)
W1bis = Isotyp(W1bis, Partition([1,1,1]))

print "V1(X)"
show(character(W1, 3, 1))

W2bis = Range(W1bis, steenrod_operators(P1, 1))

r=2
P2 = DiagonalPolynomialRing(QQ, 3, r)
op_pol = polarization_operators(P2, side="down")
#op_pol = merge(merge(op_pol, steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
W1 = PolarizedSpace(P2, W1, op_pol)
print "V1(X,Y)"
show(character(W1, 3, r))

sym_diff = {}
sym_diff2 = merge(steenrod_operators(P2, 1), steenrod_operators(P2, 2))
W2 = Range(W1, sym_diff2, add_degrees=add_degrees_isotypic)
W3 = Range(W2, sym_diff2, add_degrees=add_degrees_isotypic)

print "V2(X,Y) steenrod"
show(character(W2, 3, r, left_basis=s))
print "V3(X,Y) steenrod x2"
show(character(W3, 3, r, left_basis=s))

print "V1(X,Y)-V2(X,Y)"
charac = character(W1, 3, r) - character(W3, 3, r) - character(W2bis, 3, 1)
show(charac)

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X,Y) steenrod


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1]

V3(X,Y) steenrod x2


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] - s[2, 2] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[2, 2] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[6] # s[1, 1, 1]

In [31]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = 6
generator = {P1._grading_set((6,)) : [P1(v)]}
list_op = partial_derivatives(P1)
#list_op = merge(merge(list_op, steenrod_operators(P1, 1)), steenrod_operators(P1, 2))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1 = Isotyp(W1, Partition([1,1,1]))

print "V1(X)"
show(character(W1, 3, 1))

r=2
P2 = DiagonalPolynomialRing(QQ, 3, r)
op_pol = polarization_operators(P2, side="down")
op_pol = merge(merge(op_pol, steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
new_pol={P2._grading_set([-2,2]):[functools.partial(P2.higher_polarization, i1=0, i2=1, d1=2, d2=2)], 
         P2._grading_set([2,-2]):[functools.partial(P2.higher_polarization, i1=1, i2=0, d1=2, d2=2)]}
op_pol = merge(op_pol, new_pol)
W1 = PolarizedSpace(P2, W1, op_pol)

print "V1(X,Y)"
show(character(W1, 3, r))

list_degrees = [tuple((i,j)) for i in range(0, deg) for j in range(0, deg) if i+j>0 and i+j<deg]
sym_diff = symmetric_derivatives(P2, list_degrees)
sym_diff = merge(merge(sym_diff, steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
W2 = Range(W1, sym_diff, add_degrees=add_degrees_isotypic)

print "V2(X,Y)"
show(character(W2, 3, r))

print "V1(X,Y)-V2(X,Y)"
charac = character(W1, 3, r) - character(W2, 3, r)
show(charac)

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[3, 2] # s[1, 1, 1] + s[4] # s[1, 1, 1] + 2*s[4, 1] # s[1, 1, 1] + s[4, 2] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[5, 1] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[3, 2] # s[1, 1, 1] + s[4] # s[1, 1, 1] + 2*s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[4, 2] # s[1, 1, 1] + s[5, 1] # s[1, 1, 1] + s[6] # s[1, 1, 1]

In [42]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = 6
generator = {P1._grading_set((6,)) : [P1(v)]}
list_op1 = partial_derivatives(P1)
list_op2 = merge(steenrod_operators(P1, 1), steenrod_operators(P1, 2))
W1 = Subspace(generators=generator, operators=list_op2, add_degrees=add_degree)
W1 = Isotyp(W1, Partition([1,1,1]))
W1bis = Subspace(generators=generator, operators={}, add_degrees=add_degree)
W1bis = Isotyp(W1bis, Partition([1,1,1]))

print "V1(X)"
show(character(W1, 3, 1))

W2bis = Range(W1bis, steenrod_operators(P1, 1))

r=2
P2 = DiagonalPolynomialRing(QQ, 3, r)
op_pol = polarization_operators(P2, side="down")
op_pol = merge(merge(op_pol, steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
W1 = PolarizedSpace(P2, W1, op_pol)

print "V1(X,Y)"
show(character(W1, 3, r))

sym_diff = {}
list_degrees = [tuple((i,j)) for i in range(0, deg) for j in range(0, deg) if i+j>0 and i+j<deg]
sym_diff1 = symmetric_derivatives(P2, list_degrees)
sym_diff2 = steenrod_operators(P2, 1)
sym_diff3 = steenrod_operators(P2, 2)
sym_diff4 = merge(symmetric_derivatives(P2, list_degrees), steenrod_operators(P2, 1))
W2 = Range(W1, sym_diff2, add_degrees=add_degrees_isotypic)
W3 = Range(W2, sym_diff2, add_degrees=add_degrees_isotypic)
W4 = Range(W1, sym_diff1, add_degrees=add_degrees_isotypic)
W5 = Range(W4, sym_diff1, add_degrees=add_degrees_isotypic)

print "V2(X,Y) steenrod"
show(character(W2, 3, r, left_basis=s))
print "V3(X,Y) steenrod x2"
show(character(W3, 3, r, left_basis=s))
print "V4(X,Y) deriv sym"
show(character(W4, 3, r, left_basis=s))
print "V5(X,Y) deriv sym x2"
show(character(W5, 3, r, left_basis=s))

print "V1(X,Y)-V2(X,Y)"
charac = character(W1, 3, r) - character(W3, 3, r) - character(W4, 3, r) + character(W5, 3, r)
show(charac)

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X,Y) steenrod


...

s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1]

V5(X,Y) deriv sym x2


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[2, 2] # s[1, 1, 1] - s[3, 1] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[6] # s[1, 1, 1]

In [27]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = 6
generator = {P1._grading_set((6,)) : [P1(v)]}
list_op = {}
#list_op = partial_derivatives(P1)
#list_op = merge(merge(list_op, steenrod_operators(P1, 1)), steenrod_operators(P1, 2))
list_op = merge(list_op, steenrod_operators(P1, 1))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1 = Isotyp(W1, Partition([1,1,1]))

print "V1(X)"
show(character(W1, 3, 1))

r=2
P2 = DiagonalPolynomialRing(QQ, 3, r)
op_pol = polarization_operators(P2, max_deg=2)
#op_pol = merge(op_pol, steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
W1 = PolarizedSpace(P2, W1, op_pol)

print "V1(X,Y)"
show(character(W1, 3, r))

list_degrees = [tuple((i,j)) for i in range(0, deg) for j in range(0, deg) if i+j>0 and i+j<deg]
#sym_diff = symmetric_derivatives(P2, list_degrees)
#sym_diff = merge(merge(sym_diff, steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
sym_diff = steenrod_operators(P2, 1)
W2 = Range(W1, sym_diff, add_degrees=add_degrees_isotypic)
W3 = Range(W1, symmetric_derivatives(P2, list_degrees), add_degrees=add_degrees_isotypic )

print "V2(X,Y)"
show(character(W2, 3, r))
print "V3(X,Y)"
show(character(W3, 3, r))

print "V1(X,Y)-V2(X,Y)"
charac = character(W1, 3, r) - character(W2, 3, r)
show(charac)

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] - s[2, 2] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1]

V3(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[2, 2] # s[1, 1, 1] + s[6] # s[1, 1, 1]

On vérifie ici aussi que **$V_1(X,Y)$ stable par dérivation partielle mais pas $V_2(X,Y)$**. 

(*Ok pour les partages $3$ et $21$ mais problème avec $111$.*)

In [28]:
W12 = Subspace(W1.basis(), merge(partial_derivatives(P2), steenrod_operators(P2, 1)), add_degrees=add_degrees_isotypic)
#W12 = Isotyp2(W12, 3)
W22 = Subspace(W2.basis(), merge(partial_derivatives(P2), steenrod_operators(P2, 1)), add_degrees=add_degrees_isotypic)
#W22 = Isotyp2(W22, 3)

nu = Partition([3])
show(character(Isotyp2(W1, nu), 3, r, left_basis=s))
show(character(Isotyp2(W12, nu), 3, r, left_basis=s))
print 
show(character(Isotyp2(W2, nu), 3, r, left_basis=s))
show(character(Isotyp2(W22, nu), 3, r, left_basis=s))
#print 
#show(character(Isotyp2(W1, nu), 3, r, left_basis=m) - character(Isotyp2(W2, nu), 3, r, left_basis=m))
#show(character(Isotyp2(W12, nu), 3, r, left_basis=m) - character(Isotyp2(W22, nu), 3, r, left_basis=m))
#show(character(W12, 3, 2))
#show(character(W22, 3, 2))

AssertionError: 

Projection sur les composantes isotypiques après polarisation et ajout de tous les opérateurs **sans optimisation**. Et vérification des dimensions.  

In [29]:
r = 2
P1 = DiagonalPolynomialRing(QQ, 3, r)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = 6
generator = {P1._grading_set((6,0)) : [P1(v)]}
list_op = partial_derivatives(P1)
list_op = merge(merge(list_op, steenrod_operators(P1, 1)), steenrod_operators(P1, 2))
list_op = merge(list_op, polarization_operators(P1, max_deg=4))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

list_degrees = [tuple((i,j)) for i in range(0,deg+1) for j in range(0,deg+1) if i+j!=0 and i+j<deg+1]
sym_diff = symmetric_derivatives(P1, list_degrees)
#sym_diff = merge(merge(sym_diff, steenrod_operators(P1, 1)), steenrod_operators(P1, 2))
W2 = Range(W1, sym_diff, add_degrees=add_degrees_test)

print 'dimension', W1.dimension()
W1 = Isotyp(W1, 3)
print "V1(X,Y)"
show(character(W1, 3, r))
print 'dimension', dimension2(character(W1, 3, r), 3)

W2 = Isotyp(W2, 3)
print "V2(X,Y)"
show(character(W2, 3, r))

charac = character(W1, 3, r) - character(W2, 3, r)
print "V1(X,Y)-V2(X,Y)"
charac

dimension 152
V1(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2] # s[3] + s[2, 1] # s[1, 1, 1] + 2*s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 3*s[3] # s[2, 1] + s[3] # s[3] + 2*s[3, 1] # s[1, 1, 1] + s[3, 1] # s[2, 1] + s[4] # s[1, 1, 1] + 3*s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

dimension [((1, 1, 1), 41), ((3,), 13), ((2, 1), 49)]
V2(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2] # s[3] + s[2, 1] # s[1, 1, 1] + s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 2*s[3] # s[2, 1] + s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4] # s[2, 1] + s[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[2] # s[3] + s[2, 1] # s[2, 1] + s[3] # s[2, 1] + s[3] # s[3] + s[3, 1] # s[1, 1, 1] + s[3, 1] # s[2, 1] + 2*s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

Construction du **quotient avec des générateurs différents** : utilisant les déplacements des cases. Avec les opérateurs actuels, on retrouve bien le caractère obtenu en quotientant avant de polariser. 

In [31]:
r = 2
P1 = DiagonalPolynomialRing(QQ, 3, r)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = 6
generator = {P1._grading_set((6,0)) : [P1(v)]}
list_op = partial_derivatives(P1)
list_op = merge(merge(list_op, steenrod_operators(P1, 1)), steenrod_operators(P1, 2))
list_op = merge(list_op, polarization_operators(P1, max_deg=4))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

v014 = vandermonde(Diagram([(0,0),(1,0),(4,0)]))
generator014 = {P1._grading_set((5,0)) : [P1(v014)]}
W1_014 = Subspace(generators=generator014, operators=list_op, add_degrees=add_degree)
W1_014 = Isotyp(W1_014, 3)
print "V1_014(X,Y)"
show(character(W1_014, 3, r))

v013 = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
generator013 = {P1._grading_set((4,0)) : [P1(v013)]}
W1_013 = Subspace(generators=generator013, operators=list_op, add_degrees=add_degree)
W1_013 = Isotyp(W1_013, 3)
print "V1_013(X,Y)"
show(character(W1_013, 3, r))

v012 = vandermonde(Diagram([(0,0),(1,0),(2,0)]))
generator012 = {P1._grading_set((3,0)) : [P1(v012)]}
W1_012 = Subspace(generators=generator012, operators=list_op, add_degrees=add_degree)
W1_012 = Isotyp(W1_012, 3)
print "V1_012(X,Y)"
show(character(W1_012, 3, r))

v023 = vandermonde(Diagram([(0,0),(2,0),(3,0)]))
generator023 = {P1._grading_set((5,0)) : [P1(v023)]}
W1_023 = Subspace(generators=generator023, operators=list_op, add_degrees=add_degree)
W1_023 = Isotyp(W1_023, 3)
print "V1_023(X,Y)"
show(character(W1_023, 3, r))

generators = {P1._grading_set((5,0)) : [P1(v023), P1(v014)], P1._grading_set((4,0)) : [P1(v013)], P1._grading_set((3,0)) : [P1(v012)]}
W1_tot = Subspace(generators=generators, operators=list_op, add_degrees=add_degree)

list_degrees = [tuple((i,j)) for i in range(0,deg+1) for j in range(0,deg+1) if i+j!=0 and i+j<deg+1]
sym_diff = symmetric_derivatives(P1, list_degrees)
W2 = Range(W1, sym_diff, add_degrees=add_degrees_test)
W2_tot = Range(W1_tot, sym_diff, add_degrees=add_degrees_test)

W1 = Isotyp(W1, 3)
print "V1(X,Y)"
show(character(W1, 3, r))

W1_tot = Isotyp(W1_tot, 3)
W2_tot = Isotyp(W2_tot, 3)
print "V1_tot(X,Y)"
show(character(W1_tot, 3, r))
print "diff"
show(character(W1, 3, r) - character(W1_tot, 3, r))

V1_014(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2] # s[3] + s[2, 1] # s[1, 1, 1] + s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 2*s[3] # s[2, 1] + s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4] # s[2, 1] + s[5] # s[1, 1, 1]

V1_013(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

V1_012(X,Y)


...

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2] # s[3] + s[2, 1] # s[1, 1, 1] + 2*s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 3*s[3] # s[2, 1] + s[3] # s[3] + 2*s[3, 1] # s[1, 1, 1] + s[3, 1] # s[2, 1] + s[4] # s[1, 1, 1] + 3*s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

V1_tot(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2] # s[3] + s[2, 1] # s[1, 1, 1] + 2*s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 3*s[3] # s[2, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + 2*s[4] # s[2, 1] + 2*s[5] # s[1, 1, 1]

diff


s[3] # s[3] + s[3, 1] # s[2, 1] + s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

### Rencontre du 1er avril : $s_{22} \otimes s_{111}$

In [32]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = degree_vandermonde(Diagram([(0,0),(2,0),(4,0)]))
generator = {P1._grading_set((6,)) : [P1(v)]}
W1 = Subspace(generators=generator, operators={}, add_degrees=add_degree)
W1 = Isotyp(W1, Partition([1,1,1]))

print "V1(X)"
show(character(W1, 3, 1))

r=2
P2 = DiagonalPolynomialRing(QQ, 3, r)
op_pol = polarization_operators(P2, max_deg=4)
#op_pol = merge(op_pol, higher_polarization_operators(P2))
W1 = PolarizedSpace(P2, W1, op_pol)

print "V1(X,Y)"
show(character(W1, 3, r))

V1(X)


s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

**Vérification de l'indépendance linéaire des opérateurs.** But trouver celui qui va nous permettre d'obtenir $s_{22}$

In [33]:
P1 = DiagonalPolynomialRing(QQ, 3, 2)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
x = P1.variables()[0]
y = P1.variables()[1]

#grand vert = B
f1 = sum(y[i]*derivative(P1(v), x[i]) for i in range(0,3))
B = sum(y[i]*derivative(P1(f1), x[i], 3) for i in range(0,3))

#petit vert
f3 = sum(y[i]*derivative(P1(v), x[i], 2) for i in range(0,3))
Z = sum(y[i]*derivative(P1(f3), x[i], 2) for i in range(0,3))

#rouge = A
f5 = sum(y[i]*derivative(P1(v), x[i]) for i in range(0,3))
f6 = sum(derivative(P1(f5), x[i]) for i in range(0,3))
A = sum(y[i]*derivative(P1(f6), x[i], 2) for i in range(0,3))

#bleu = C
f8 = sum(derivative(P1(v), x[i]) for i in range(0,3))
f9 = sum(derivative(P1(f8), x[i]) for i in range(0,3))
f10 = sum(y[i]*derivative(P1(f9), x[i]) for i in range(0,3))
C = sum(y[i]*derivative(P1(f10), x[i]) for i in range(0,3))

g1 = sum(x[i]*derivative(P1(v), x[i], 2) for i in range(0,3))
g2 = sum(y[i]*derivative(P1(g1), x[i], 2) for i in range(0,3))
D = sum(y[i]*derivative(P1(g2), x[i]) for i in range(0,3))

#test element
g3 = sum(y[i]*derivative(P1(v), x[i], 2) for i in range(0,3))
E = sum(x[i]*y[i]*derivative(P1(g3), x[i], 3) for i in range(0,3))

show(E)

S = Subspace([A, B, C, D, E])
show(S.basis())
S.dimension()

-48*x00^2*x10*x11 + 48*x01^2*x10*x11 + 48*x00^2*x10*x12 - 48*x02^2*x10*x12 - 48*x01^2*x11*x12 + 48*x02^2*x11*x12

{0: (-x00*x01*x10^2 + x00*x02*x10^2 + x00*x01*x11^2 - x01*x02*x11^2 - x00*x02*x12^2 + x01*x02*x12^2,
  -x01^2*x10^2 + x02^2*x10^2 + x00^2*x11^2 - x02^2*x11^2 - x00^2*x12^2 + x01^2*x12^2,
  x00^2*x10*x11 - x01^2*x10*x11 - x00^2*x10*x12 + x02^2*x10*x12 + x01^2*x11*x12 - x02^2*x11*x12)}

3